In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("C:\\Users\\Sarrang\\p2p\\treated.csv")

In [3]:
df.shape

(176563, 88)

In [4]:
df.columns.get_loc('Rating')

41

In [5]:
# want to exclude columns 38 and 41
all_columns = list(range(df.shape[1]))

# Exclude columns 38 and 41 which are probability of default and rating
exclude_columns = [38, 41]
included_columns = [col_idx for col_idx in all_columns if col_idx not in exclude_columns]

X= df.iloc[:, included_columns]
df_idx=df.iloc[:, included_columns]

### experimenting with rating as dependent variable 

In [6]:
y=df.iloc[:,41]

In [7]:
 # excluded 'ProbabilityOfDefault'
numerical=['BidsManual',
 'AppliedAmount',
 'Amount',
 'Interest',
 'MonthlyPayment',
 'OccupationArea',
 'HomeOwnershipType',
 'IncomeFromPrincipalEmployer',
 'IncomeFromPension',
 'IncomeFromFamilyAllowance',
 'IncomeFromSocialWelfare',
 'IncomeFromLeavePay',
 'IncomeFromChildSupport',
 'IncomeOther',
 'IncomeTotal',
 'LiabilitiesTotal',
 'DebtToIncome',
 'FreeCash',
 'LossGivenDefault',
 'ExpectedReturn',
 'PrincipalOverdueBySchedule',
 'PrincipalPaymentsMade',
 'InterestAndPenaltyPaymentsMade',
 'PrincipalBalance',
 'InterestAndPenaltyBalance',
 'NoOfPreviousLoansBeforeLoan',
 'AmountOfPreviousLoansBeforeLoan',
 'PreviousEarlyRepaymentsCountBeforeLoan',
 'ApplicationSignedHour',
 'ApplicationSignedWeekday',
 'LanguageCode',
 'Age',
 'LoanDuration',
 'UseOfLoan',
 'ExistingLiabilities',
 'RefinanceLiabilities',
 'MonthlyPaymentDay',
 'CreditScoreEsMicroL',
 'LoanWaitTime',
 'PaymentToLoanDateDiff',
 'LoanTerm']

In [8]:
nloc = []

for feature in numerical:
    try:
        idx = X.columns.get_loc(feature)
        nloc.append(idx)
    except ValueError:
        print(f"Column '{feature}' not found in X columns.")


In [9]:
X=X.values
y=y.values

## Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [11]:
from sklearn.preprocessing import PowerTransformer

# Initialize PowerTransformer
power_transformer = PowerTransformer()

# Fit the transformer on training data and transform training data
X_train[:, nloc] = power_transformer.fit_transform(X_train[:, nloc])

# Transform testing data using the fitted transformer from training data
X_test[:, nloc] = power_transformer.transform(X_test[:, nloc])


In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define the hyperparameter search space
param_dist = {
    'max_rounds': randint(50, 200),
    'learning_rate': uniform(0.001, 0.1),
    'max_bins': randint(50, 255),
    'max_interaction_bins': randint(5, 20),
    'interactions': randint(5, 20),
    'outer_bags': randint(5, 20),
    'inner_bags': randint(0, 10),
    'random_state': randint(0, 100)
}

# Instantiate the EBM classifier
ebm_classifier = ExplainableBoostingClassifier()

# Instantiate RandomizedSearchCV
random_search_classifier = RandomizedSearchCV(
    estimator=ebm_classifier,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    scoring='accuracy',  # Scoring metric for classification
    cv=3,  # Cross-validation folds
    verbose=2,  # Controls the verbosity
    n_jobs=-1,  # Use all available processors
    random_state=42  # Random seed for reproducibility
)

# Perform hyperparameter tuning
random_search_classifier.fit(X_train, y_train)

# Get best parameters and best model
best_params_classifier = random_search_classifier.best_params_
best_model_classifier = random_search_classifier.best_estimator_

print("Best Parameters (Classification):", best_params_classifier)
print("Best Model (Classification):", best_model_classifier)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
best_model_classifier

In [19]:
ebc=ExplainableBoostingClassifier()#ebm with default params is yeilding better results
ebc.fit(X_train,y_train)

ExplainableBoostingRegressor()

In [20]:
y_pred=ebc.predict(X_test)

In [ ]:
from joblib import dump
dump(xgbr,'p2p_ebm_cls.joblib')